In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Thursday, August 22, 2024
Last Modified: Thursday, September 4, 2025
Program Purpose: To create most of the main plots in the paper, such as the grid cell, lat, and the maps of the different biome definitions.
This program also reports the different IAV values and ranges.
"""

In [ ]:
#Importing appropriate libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import statistics as stats
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
#Making font times
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block handles the IAV calculation for the gridcell based IAV analysis

# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

#The ordered dict to keep the values with the associated models in the correct order
from collections import OrderedDict
all_datasets_IAV = OrderedDict()

# Loading and processing all of the files
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value of adjustment
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented
    dataset.close()

    # Removing the wrongly high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero

    # Removing the water values from the yibs that are not blocked out
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0) #Making across the 52 years
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=(data_modis_52 == 0)) #Masking the water out that shouldn't be there

    # Finding the average flux of each grid cell over the 52 years.
    overall_mean_flux_by_grid = terrestrial_flux.mean(axis=0)

    # Finding the flux anomaly for each cell for every year.
    flux_anomaly_yearly_by_grid = terrestrial_flux - overall_mean_flux_by_grid

    # Calculating the yearly global flux anomay by summing all of the individual flux anomalies for each grid cell for that year.
    yearly_global_flux_anomaly = [] #List of the global flux anomalies
    for i in range(52): #The 52 years of anomalies
        yearly_global_flux_anomaly.append(np.sum(flux_anomaly_yearly_by_grid[i,:,:]))

    # Finding the bottom sum for the equation (absolute value of global flux anomaly over the years)
    bottom_sum = np.sum(np.abs(yearly_global_flux_anomaly))

    # New array to add top values to
    placeholder_array_top_values = np.empty_like(terrestrial_flux)

    for i in range(52):
        # Avoiding division by zero
        if yearly_global_flux_anomaly[i] != 0:
            placeholder_array_top_values[i] = flux_anomaly_yearly_by_grid[i] * (np.abs(yearly_global_flux_anomaly[i]) / yearly_global_flux_anomaly[i])
        else:
            placeholder_array_top_values[i] = 0 #Setting to zero if case of division by zero

    summed_top_values = np.sum(placeholder_array_top_values, axis=0)  # Summing the values for the top of the array

    placeholder_array = summed_top_values / bottom_sum  # Dividing the top and bottom of the equation

    # Adding the placeholder array to the list of IAV maps
    all_datasets_IAV[file_name] = placeholder_array


### Getting the mean and the median values
list_data = []
for keys in all_datasets_IAV:
    list_data.append(all_datasets_IAV[keys])

#Stacking
stacked_data = np.stack(list_data)

#Compiling the mean and median
mean_data = np.ma.mean(stacked_data, axis=0)
median_data = np.ma.median(stacked_data, axis=0)

#Multiplying all values by 100 in the mean and median data such that we have IAV represented as %
mean_data = mean_data*100
median_data = median_data*100

#Checking scaling
print(f"The sum of the mean map after *100: {np.sum(mean_data)}")
print(f"The sum of the median map after *100: {np.sum(median_data)}") #Since we are taking the median we will not get a sum to 100% IAV!!!!!!!!!!!!!!

## Printing what the mean and median look like
plt.imshow(mean_data)
plt.show()
plt.imshow(median_data)
plt.show()




In [ ]:
#This block handles the mapping of the median data on a global scale 

#Getting the min and max for the colorbar
final_max = np.nanmax(median_data)
final_min = np.nanmin(median_data)

#Making the middle 0, so that the colorbar can be split on the colors
norm = mcolors.TwoSlopeNorm(vmin = final_min, vcenter= 0, vmax= final_max)

#Plotting the figure
fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={'projection': ccrs.PlateCarree()})
im = ax.imshow(median_data, norm=norm, cmap='seismic', origin='upper', extent=[-180, 180, -90, 90])


#Adding the borders and coast
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Adding the colorbar to the left
cb = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02)

#Tweaking the colorbar appearance
cb.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='y')
cb.ax.set_position([ax.get_position().x0 - 0.02, ax.get_position().y0, 0.02, ax.get_position().height])
cb.ax.yaxis.offsetText.set_fontsize(60)  
cb.set_label('IAV Contribution (%)', fontsize=60)
cb.ax.yaxis.set_ticks_position('left')
cb.ax.yaxis.set_label_position('left')
cb.ax.tick_params(labelsize=56, width=6)

#Sbow plot
plt.show()


In [ ]:
#This block handles the plotting of the latitudinal plot

#Masking out the water area along each band so that the water does not influence the calculation of mean/median contirbution
# Importing the MODIS Map to remove the water.
modis_data = np.load("MODIS_Map.npy")
temp_median_data = median_data
temp_median_data[modis_data == 0] = np.nan #making the water into nan to prevent its inclusion from the mean
latitudinal_band = np.nanmean(median_data, axis=1) #Taking the mean value across the axis


#Because of empty slices due to no land at the lower bounds, setting these values to 0 since no IAV contribution
latitudinal_band[np.isnan(latitudinal_band)] = 0
#Confirming the shape of the latitudinal data
print(f"The shape of the data: {np.shape(latitudinal_band)}")

#Creating a linechart that has the latitudinal values as the one axis and the median IAV the other
#Filling up the lat bands such that the the plot can be shown on its side
latBands = []
startVal = 90
for i in range(360):
    latBands.append(startVal)
    startVal-=0.5

#Plotting the latitudinal plot
fig, ax = plt.subplots(figsize=(10,10))
plt.plot(latitudinal_band, latBands,  color='black', linewidth=5)
plt.xlabel("IAV Contribution (%)", fontsize=40)
plt.ylabel("Latitude", fontsize=40)
plt.ticklabel_format(axis='x', scilimits=(0,0), style='sci')
ax.xaxis.offsetText.set_fontsize(28)
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
ax.invert_xaxis()
plt.tick_params(labelsize=28, width=6)
plt.tick_params(axis='both', labelsize=28)
plt.yticks([-90, -60, -30, 0, 30, 60, 90])
plt.ylim(-90,90)

#making the axes thicker
ax.spines['right'].set_linewidth(1)
ax.spines['left'].set_linewidth(1)
ax.spines['top'].set_linewidth(1)
ax.spines['bottom'].set_linewidth(1)

#Printing out the values at the differet lat bands
count = 0
for value in latitudinal_band:
    print(f"LatBand: {latBands[count]} IAV: {value}")
    count += 1

print(f"The largest contribution: {np.max(latitudinal_band)}")




In [ ]:
# This block handles the creation of the plot containing both the gridcell map and the lat plot aligned next to one another (desired fig)

fig = plt.figure(figsize=(50,20))
spec = gridspec.GridSpec(ncols=2, nrows=2, figure = fig, width_ratios=(4, 1), height_ratios=(20,1))

#Handling the map
fig_ax2 = fig.add_subplot(spec[0,0], projection = ccrs.PlateCarree())
fig_ax2.set_title("   (a)", fontsize=70, loc='left', fontweight='bold')
im = fig_ax2.imshow(median_data, norm=norm, cmap='seismic', origin='upper', extent=[-180, 180, -90, 90])
fig_ax2.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded
fig_ax2.add_feature(cfeature.COASTLINE)
fig_ax2.add_feature(cfeature.BORDERS, linestyle=':')
gl = fig_ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.ylocator = mticker.FixedLocator([-60, -30, 0, 30, 60, 90])
gl.ylabel_style = {'size': 60}
gl.xlabel_style = {'size': 45}

#Handling the colorbar
fig_ax1 = fig.add_subplot(spec[1,0])
cb = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='seismic'), cax=fig_ax1, orientation='horizontal')
cb.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='x')
cb.ax.xaxis.offsetText.set_fontsize(60)  
cb.set_label('IAV Contribution (%)', fontsize=60)
cb.ax.yaxis.set_ticks_position('left')
cb.ax.yaxis.set_label_position('left')
cb.ax.tick_params(labelsize=56, width=6)

#Handling the line plot
fig_ax3 = fig.add_subplot(spec[0,1])

fig_ax3.plot(latitudinal_band, latBands,  color='black', linewidth=5)
fig_ax3.set_title("(b)", fontsize=70, loc='left', fontweight='bold')
fig_ax3.set_ylim(-60,90)
fig_ax3.set_yticks([])
fig_ax3.axhline(y=-30, linewidth=2, color='gray', alpha=0.5, linestyle='--')
fig_ax3.axhline(y=0, linewidth=2, color='gray', alpha=0.5, linestyle='--')
fig_ax3.axhline(y=30, linewidth=2, color='gray', alpha=0.5, linestyle='--')
fig_ax3.axhline(y=60, linewidth=2, color='gray', alpha=0.5, linestyle='--')
fig_ax3.set_xlabel("IAV\nContribution (%)", fontsize=60)
fig_ax3.yaxis.set_label_position('right')
fig_ax3.set_ylabel("Latitude", fontsize=60,)
fig_ax3.ticklabel_format(axis='x', scilimits=(0,0), style='sci')
fig_ax3.tick_params(labelsize=60)
fig_ax3.xaxis.offsetText.set_fontsize(60)

#Making the plot tight to save space
plt.tight_layout()

In [ ]:
#This block handles the creation of the AI and PANGEA map plot, and the subsequently proper masking that allows the appropriate IAV calcs

# Only using the ARID range of arid to dry subhumid, excluding hyperarid as this will lead to inclusion of deserts
pangea_array = np.load('aez_tropical_PANGEA.npy')
arid_array = np.load("halfdeg_AI.npy")

# Making the PANGEA array all == 1 where there is actual tropical area
pangea_array[(pangea_array != 5) & (pangea_array != 6)& (pangea_array != 11) & (pangea_array != 12) & (pangea_array != 52)] = 0
pangea_array[pangea_array != 0] = 1

# Making the ARID array all == 2 where there is actual dryland area
arid_array[(0.05 < arid_array) & (arid_array< 0.65)]= -2
arid_array[arid_array != -2] = 0
arid_array[arid_array == -2] = 2
arid_array[300:,:] = 0 # Removing Antarctica as this regions doesn't have flux values associated

# Loading in the MODIS for water mask
modis_data = np.load("MODIS_Map.npy")
# Puting the 2 arrays together by giving any overlapping priority to the tropical regions
combined_array = np.zeros((360,720))
combined_array[arid_array == 2] = 2 # Setting arid to 2
combined_array[pangea_array == 1] = 1 # setting tropical to 1
combined_array[(modis_data != 0) & (arid_array != 2) & (pangea_array != 1)] = 3 # Setting the other land to 3
combined_array[modis_data == 0] = 0


# Seeing what this looks like projected
# PANGEA and ARID map

# This is the plot where the groups are based off of the ARID dryland and the PANGEA tropical respectively
fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={'projection': ccrs.PlateCarree()})
cmap = ListedColormap([ "white", "darkgreen", "orange", "white"]) #Making the colormap
im = ax.imshow(combined_array, cmap=cmap, origin='upper', extent=[-180, 180, -90, 90])
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

#Adding the legend
drylandBar = mpatches.Patch(facecolor='orange', label='Dryland', edgecolor='black')
tropicalBar = mpatches.Patch(facecolor='darkgreen', label='Tropical\nForest', edgecolor='black')
#plt.legend(handles=[drylandBar, tropicalBar], loc='lower left', bbox_to_anchor = [0, 0.1], fontsize=85)

#Adding the title for the biome definition
plt.title("(c)\nAridity Index & Agro-Ecological Zones", loc="left", fontsize=85, fontweight='bold')

#Borders and coastlines
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Show plot tight
plt.tight_layout()
plt.show()

In [ ]:
#This block handles the IAV calculations for the PANGEA and ARID def

all_tropical_IAV_PANGEA = OrderedDict() # Tropical IAV
all_dryland_IAV_ARID = OrderedDict() # Dryland IAV
all_other_IAV_PA = OrderedDict() # other IAV

# Expanding the map to be for 52 years to match with the flux data
combined_maps_PA = np.tile(combined_array, (52,1,1))

# Loading and processing all of the files
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the undue high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    # Find the tropical flux each year.
    # Add the values from terrestrial_flux where combined_array == 1
    terrestrial_flux_tropical = np.ma.array(terrestrial_flux, mask = (combined_maps_PA != 1)) # ==1 for the tropical region
    yearly_fluxes_tropical = terrestrial_flux_tropical.sum(axis=(1,2)) # Yearly flux from the tropical regions
    yearly_fluxes_tropical_list = yearly_fluxes_tropical.tolist()
    # Calculate yearly tropical flux anomaly.
    tropical_average_flux = sum(yearly_fluxes_tropical_list) / len(yearly_fluxes_tropical_list) # Sum annual fluxes / # years
    yearly_tropical_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_tropical_list:
        anomaly_to_add = yearly_flux - tropical_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_tropical_flux_anomaly.append(anomaly_to_add)

    # Find the dryland flux each year.
    # Add the values from terrestrial_flux where combined_array == 2
    terrestrial_flux_dryland = np.ma.array(terrestrial_flux, mask = (combined_maps_PA  != 2)) # ==2 for the dryland region
    yearly_fluxes_dryland = terrestrial_flux_dryland.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_dryland_list = yearly_fluxes_dryland.tolist()

    # Calculate yearly dryland flux anomaly.
    dryland_average_flux = sum(yearly_fluxes_dryland_list) / len(yearly_fluxes_dryland_list) # Sum annual fluxes / # years
    yearly_dryland_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_dryland_list:
        anomaly_to_add = yearly_flux - dryland_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_dryland_flux_anomaly.append(anomaly_to_add)

    # Fnding the flux of all other regions every year.
    # Adding values terrestrial_flux where combined_maps_52 is not water, tropical, or dryland
    terrestrial_flux_other = np.ma.array(terrestrial_flux, mask = ((combined_maps_PA == 0) | (combined_maps_PA== 1) | (combined_maps_PA == 2)))
    yearly_fluxes_other = terrestrial_flux_other.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_other_list = yearly_fluxes_other.tolist()

    # Calculate yearly other flux anomaly.
    other_average_flux = sum(yearly_fluxes_other_list) / len(yearly_fluxes_other_list) # Sum annual fluxes / # years
    yearly_other_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_other_list:
        anomaly_to_add = yearly_flux - other_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_other_flux_anomaly.append(anomaly_to_add)

    # Calculating the yearly global flux anomlay now that we have the flux anomaly of the 3 regions (Tropical, Dryland, and other)
    yearly_global_flux_anomaly = list()
    for i in range(52):
        yearly_global_flux_anomaly.append(yearly_tropical_flux_anomaly[i] + yearly_dryland_flux_anomaly[i] + yearly_other_flux_anomaly[i])


    # Calculating the tropical IAV values.
    top_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_tropical += (yearly_tropical_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_tropical += abs(yearly_global_flux_anomaly[i])
    # Adding the tropical IAV for that model to the master dictionary.
    all_tropical_IAV_PANGEA[file_name] = top_sum_tropical / bottom_sum_tropical * 100

    # Calculating the dryland IAV values.
    top_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_dryland += (yearly_dryland_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_dryland += abs(yearly_global_flux_anomaly[i])
    # Adding the dryland IAV for that model to the master dictionary.
    all_dryland_IAV_ARID[file_name] = top_sum_dryland / bottom_sum_dryland * 100

    # Calculating the other IAV values.
    top_sum_other = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_other += (yearly_other_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_other = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_other += abs(yearly_global_flux_anomaly[i])
    # Adding the other IAV for that model to the master dictionary.
    all_other_IAV_PA[file_name] = top_sum_other / bottom_sum_other * 100

#Ensuring sum to 100
print(f"Koppen sum= {stats.mean(all_dryland_IAV_ARID.values()) + stats.mean(all_tropical_IAV_PANGEA.values()) + stats.mean(all_other_IAV_PA.values())}")


In [ ]:
#This block handles the IAV calc for the Koppen based defintion

# IAV by region
all_tropical_IAV_KG = OrderedDict() # Tropical IAV
all_dryland_IAV_KG = OrderedDict() # Dryland IAV
all_other_IAV_KG = OrderedDict()

# Creating the map of the K&G definitions
file_koppen = nc.Dataset("KG_360x720.nc", "r")
climate_names = file_koppen.variables['class_name'][:]
climate_type = file_koppen.variables['kg'][:]
print(f" The order of K&G Classes {climate_names}")
file_koppen.close()

#0 and 1 is tropical forest and monsoon && 2 and 3 are svannas and 4 and 5 are the 2 semi-arid regions
# Getting the individual tropical and dryland climate classificatons
tropic_climate = climate_type[:2, :, :]
tropic_climate_2d = np.sum(tropic_climate, axis=0)
dryland_climate = climate_type[2:6, :, :] 
dryland_climate_2d = np.sum(dryland_climate, axis=0)

# Removing instances where less than 50% of dryland or tropical, as setting to zero
mask_50_tropic = tropic_climate_2d > 0.5
mask_50_dryland = dryland_climate_2d > 0.5
tropic_climate_2d[~mask_50_tropic] = 0
dryland_climate_2d[~mask_50_dryland] = 0

combined_maps = np.zeros((360,720))

combined_maps[tropic_climate_2d>0] = 2 # Combined_maps == 2 to access the tropical region
combined_maps[dryland_climate_2d>0] = 3 # Combined_maps == 3 to access the dryland region
combined_maps[modis_data == 0] = 1 # Water


combined_maps_52 = np.tile(combined_maps, (52,1,1)) # Tiling to be over 52 years

# Loading all of the files and processing
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the undue high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
            
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)

        mask = data_modis_52 != 0

        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)


    # Find the tropical flux each year.
    # Add the values from terrestrial_flux where combined_maps_52 == 2
    terrestrial_flux_tropical = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 2)) # ==2 for the tropical region
    yearly_fluxes_tropical = terrestrial_flux_tropical.sum(axis=(1,2)) # Yearly flux from the tropical regions
    yearly_fluxes_tropical_list = yearly_fluxes_tropical.tolist()
    # Calculate yearly tropical flux anomaly.
    tropical_average_flux = sum(yearly_fluxes_tropical_list) / len(yearly_fluxes_tropical_list) # Sum annual fluxes / # years
    yearly_tropical_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_tropical_list:
        anomaly_to_add = yearly_flux - tropical_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_tropical_flux_anomaly.append(anomaly_to_add)
    
    # Find the dryland flux each year.
    # Add the values from terrestrial_flux where combined_maps_52 == 2
    terrestrial_flux_dryland = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 3)) # ==3 for the dryland region
    yearly_fluxes_dryland = terrestrial_flux_dryland.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_dryland_list = yearly_fluxes_dryland.tolist()
    
    # Calculate yearly dryland flux anomaly.
    dryland_average_flux = sum(yearly_fluxes_dryland_list) / len(yearly_fluxes_dryland_list) # Sum annual fluxes / # years
    yearly_dryland_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_dryland_list:
        anomaly_to_add = yearly_flux - dryland_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_dryland_flux_anomaly.append(anomaly_to_add)

    # Fnding the flux of all other regions every year.
    # Adding values terrestrial_flux where combined_maps_52 is not water, tropical, or dryland
    terrestrial_flux_other = np.ma.array(terrestrial_flux, mask = ((combined_maps_52 == 1) | (combined_maps_52 == 2) | (combined_maps_52 == 3)))
    yearly_fluxes_other = terrestrial_flux_other.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_other_list = yearly_fluxes_other.tolist()

    

    # Calculate yearly other flux anomaly.
    other_average_flux = sum(yearly_fluxes_other_list) / len(yearly_fluxes_other_list) # Sum annual fluxes / # years
    yearly_other_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_other_list:
        anomaly_to_add = yearly_flux - other_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_other_flux_anomaly.append(anomaly_to_add)

    # Calculating the yearly_global_flux_anomaly now that we have the flux anomaly of the 3 regions (Tropical, Dryland, and other)
    yearly_global_flux_anomaly = list()
    for i in range(52):
        yearly_global_flux_anomaly.append(yearly_tropical_flux_anomaly[i] + yearly_dryland_flux_anomaly[i] + yearly_other_flux_anomaly[i])

    
    # Calculating the tropical IAV values.
    top_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_tropical += (yearly_tropical_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_tropical += abs(yearly_global_flux_anomaly[i])
    # Adding the tropical IAV for that model to the master dictionary.
    all_tropical_IAV_KG[file_name] = top_sum_tropical / bottom_sum_tropical * 100

    # Calculating the dryland IAV values.
    top_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_dryland += (yearly_dryland_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_dryland += abs(yearly_global_flux_anomaly[i])
    # Adding the dryland IAV for that model to the master dictionary.
    all_dryland_IAV_KG[file_name] = top_sum_dryland / bottom_sum_dryland * 100

    # Calculating the other IAV values.
    top_sum_other = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_other += (yearly_other_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_other = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_other += abs(yearly_global_flux_anomaly[i])
    # Adding the other IAV for that model to the master dictionary.
    all_other_IAV_KG[file_name] = top_sum_other / bottom_sum_other * 100

#Verify sum to 100
print(f"Koppen sum= {stats.mean(all_dryland_IAV_KG.values()) + stats.mean(all_tropical_IAV_KG.values()) + stats.mean(all_other_IAV_KG.values())}")

In [ ]:
#Doing the plotting of the Koppen map in this block
fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={'projection': ccrs.PlateCarree()})
cmap = ListedColormap([ "white", "white", "darkgreen", "orange"]) #Making the colormap
im = ax.imshow(combined_maps, cmap=cmap, origin='upper', extent=[-180, 180, -90, 90])
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

#Adding the legend
drylandBar = mpatches.Patch(facecolor='orange', label='Dryland', edgecolor='black')
tropicalBar = mpatches.Patch(facecolor='darkgreen', label='Tropical\nForest', edgecolor='black')
plt.legend(handles=[drylandBar, tropicalBar], loc='lower left', bbox_to_anchor = [0, 0.1], fontsize=85)

#Adding the title for the biome definition
plt.title("(a)\nKöppen", loc="left", fontsize=85, fontweight='bold')

#Borders and coastlines
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
#This block handles the MODIS based IAV calculation

# Importing the MODIS Map to work with.
modis_data = np.load("MODIS_Map.npy")

# Get the individual tropical and dryland climate classificatons.
tropic_mask = modis_data == 1
tropic_climate = modis_data.copy()
tropic_climate[~tropic_mask] = 0

dryland_mask = modis_data == 3
dryland_climate = modis_data.copy()
dryland_climate[~dryland_mask] = 0

combined_maps = np.zeros((360,720))
combined_maps[modis_data == 0] = 1 # == 1 for the water 
combined_maps[tropic_climate>0] = 2 # Combined_maps == 2 to access the tropical region
combined_maps[dryland_climate>0] = 3 # Combined_maps == 3 to access the dryland region

plt.subplots(figsize=(20,10))
plt.imshow(combined_maps)
plt.colorbar()
plt.show()

# Expanding the map to be for 52 years to match with the flux data
combined_maps_52 = np.tile(combined_maps, (52,1,1))
print(f"The shape of combined_maps_52 after tiling: {np.shape(combined_maps_52)}")

# Getting how many cells are in the dryland definition.
copy_array = combined_maps.copy()
copy_array_masked = np.ma.array(copy_array, mask = (combined_maps != 3))
area_dry_modis_raw = np.sum(area_file * copy_array_masked)

copy_array = combined_maps.copy()
copy_array_masked = np.ma.array(copy_array, mask = (combined_maps != 0)) # Other land
copy_array_masked[copy_array_masked ==0] = 1
area_other_land_modis = np.sum(area_file * copy_array_masked)

copy_array = combined_maps.copy()
copy_array_masked = np.ma.array(copy_array, mask = (combined_maps != 2)) # Tropical
area_tropical_modis = np.sum(area_file * (copy_array_masked / 2))

total_area = area_dry_modis_raw + area_other_land_modis + area_tropical_modis

area_dry_modis = (area_dry_modis_raw / total_area) * 100  #Converting to the percentage value.



# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')


from collections import OrderedDict
all_tropical_IAV = OrderedDict() # Tropical IAV
all_dryland_IAV = OrderedDict() # Dryland IAV
all_other_IAV = OrderedDict()

all_land_yearly_flux = OrderedDict() # Global carbon flux (sum flux)
all_tropical_yearly_flux = OrderedDict() # Tropical carbon flux (sum flux)
all_dryland_yearly_flux = OrderedDict() # Dryland carbon flux (sum flux)

all_land_yearly_anomaly = OrderedDict() # Global carbon flux anomaly (annual sum flux - mean of annual sum fluxes)
all_tropical_yearly_anomaly = OrderedDict() # Tropical carbon flux anomaly
all_dryland_yearly_anomaly = OrderedDict() # Dryland carbon flux anomaly



# Load and process all of the files.
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the unduely high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out 
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    # Find the tropical flux each year.
    # Add the values from terrestrial_flux where combined_maps_52 == 2
    terrestrial_flux_tropical = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 2)) # ==2 for the tropical region
    yearly_fluxes_tropical = terrestrial_flux_tropical.sum(axis=(1,2)) # Yearly flux from the tropical regions
    yearly_fluxes_tropical_list = yearly_fluxes_tropical.tolist()
    # Calculate yearly tropical flux anomaly.
    tropical_average_flux = sum(yearly_fluxes_tropical_list) / len(yearly_fluxes_tropical_list) # Sum annual fluxes / # years
    yearly_tropical_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_tropical_list:
        anomaly_to_add = yearly_flux - tropical_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_tropical_flux_anomaly.append(anomaly_to_add)

    # Find the dryland flux each year.
    # Add the values from terrestrial_flux where combined_maps_52 == 2
    terrestrial_flux_dryland = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 3)) # ==3 for the dryland region
    yearly_fluxes_dryland = terrestrial_flux_dryland.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_dryland_list = yearly_fluxes_dryland.tolist()

    # Calculate yearly dryland flux anomaly.
    dryland_average_flux = sum(yearly_fluxes_dryland_list) / len(yearly_fluxes_dryland_list) # Sum annual fluxes / # years
    yearly_dryland_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_dryland_list:
        anomaly_to_add = yearly_flux - dryland_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_dryland_flux_anomaly.append(anomaly_to_add)

    # Fnding the flux of all other regions every year.
    # Adding values terrestrial_flux where combined_maps_52 is not water, tropical, or dryland
    terrestrial_flux_other = np.ma.array(terrestrial_flux, mask = ((combined_maps_52 == 1) | (combined_maps_52 == 2) | (combined_maps_52 == 3)))
    yearly_fluxes_other = terrestrial_flux_other.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_other_list = yearly_fluxes_other.tolist()

    # Calculate yearly other flux anomaly.
    other_average_flux = sum(yearly_fluxes_other_list) / len(yearly_fluxes_other_list) # Sum annual fluxes / # years
    yearly_other_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_other_list:
        anomaly_to_add = yearly_flux - other_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_other_flux_anomaly.append(anomaly_to_add)

    # Calculating the yearly_global_flux_anomaly now that we have the flux anomaly of the 3 regions (Tropical, Dryland, and other)
    yearly_global_flux_anomaly = list()
    for i in range(52):
        yearly_global_flux_anomaly.append(yearly_tropical_flux_anomaly[i] + yearly_dryland_flux_anomaly[i] + yearly_other_flux_anomaly[i])

    # Calculating the tropical IAV values.
    top_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_tropical += (yearly_tropical_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_tropical += abs(yearly_global_flux_anomaly[i])
    # Adding the tropical IAV for that model to the master dictionary.
    all_tropical_IAV[file_name] = top_sum_tropical / bottom_sum_tropical *100

    # Calculating the dryland IAV values.
    top_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_dryland += (yearly_dryland_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_dryland += abs(yearly_global_flux_anomaly[i])
    # Adding the dryland IAV for that model to the master dictionary.
    all_dryland_IAV[file_name] = top_sum_dryland / bottom_sum_dryland *100

    # Calculating the other IAV values.
    top_sum_other = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_other += (yearly_other_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_other = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_other += abs(yearly_global_flux_anomaly[i])
    # Adding the other IAV for that model to the master dictionary.
    all_other_IAV[file_name] = top_sum_other / bottom_sum_other *100


#### Saving the dryland IAV values as MODIS specific to use lower in the file for a comparison between the different ways of classifying IAV 
all_dryland_IAV_MODIS = all_dryland_IAV
all_other_IAV_MODIS = all_other_IAV
all_tropical_IAV_MODIS= all_tropical_IAV

#Verifying sum to 100
print(f"Koppen sum= {stats.mean(all_dryland_IAV_MODIS.values()) + stats.mean(all_tropical_IAV_MODIS.values()) + stats.mean(all_other_IAV_MODIS.values())}")

In [ ]:
#Making map of the MODIS region in this block
fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={'projection': ccrs.PlateCarree()})
cmap = ListedColormap([ "white", "white", "darkgreen", "orange"]) #Making the colormap
im = ax.imshow(combined_maps, cmap=cmap, origin='upper', extent=[-180, 180, -90, 90])
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

#Adding the legend
drylandBar = mpatches.Patch(facecolor='orange', label='Dryland', edgecolor='black')
tropicalBar = mpatches.Patch(facecolor='darkgreen', label='Tropical\nForest', edgecolor='black')
#plt.legend(handles=[drylandBar, tropicalBar], loc='lower left', bbox_to_anchor = [0, 0.1], fontsize=85)

#Adding the title for the biome definition
plt.title("(b)\nMODIS", loc="left", fontsize=85, fontweight='bold')

#Borders and coastlines
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Show plot
plt.tight_layout()
plt.show()



In [ ]:
#Making a new boxplot with all of the different biome defintions in the same plot

# Creating a list of all of the data
data = [list(all_dryland_IAV_KG.values()), list(all_tropical_IAV_KG.values()), list(all_dryland_IAV_MODIS.values()), list(all_tropical_IAV_MODIS.values()), list(all_dryland_IAV_ARID.values()), list(all_tropical_IAV_PANGEA.values())]

#Adjusting the figure size
plt.figure(figsize=(3939/100, 1593/100), dpi=100)

# Creating the boxplot
boxplots = plt.boxplot(data, positions=[1, 2, 3.5, 4.5, 6, 7], widths=0.9, patch_artist=True, medianprops=dict(color='black', linewidth=8), showmeans=True,
    meanprops={"marker": "D", "markerfacecolor": "grey", "markeredgecolor": "black", "markersize": 32},
    flierprops=dict( markersize=24, markeredgewidth=10), capprops=dict(linewidth=10), whiskerprops=dict(linewidth=10),  boxprops=dict(linewidth=10)
)

#Alternating colors
colors = ['orange', 'darkgreen', 'orange', 'darkgreen', 'orange', 'darkgreen']

boxes = boxplots['boxes']
whiskers = boxplots['whiskers']
caps = boxplots['caps']
fliers = boxplots['fliers']
medians = boxplots['medians']

i = 0
for box in boxes:
    box.set_facecolor('white')
    box.set_edgecolor(colors[i])
    i+=1

i = 0
for median in medians:
    median.set_color(colors[i])
    i+=1


i = 0
j=0
for whisker in whiskers:
    whisker.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

i = 0
j=0
for cap in caps:
    cap.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

fliers[0].set(markerfacecolor = 'white', markeredgecolor=colors[0])
fliers[4].set(markerfacecolor = 'white', markeredgecolor=colors[4])
fliers[5].set(markerfacecolor = 'white', markeredgecolor=colors[5])

#Adding labels
plt.xticks([1.5, 4, 6.5], ['Köppen', 'MODIS', 'Aridity Index &\nAgro-Ecological Zones'], fontsize=70)
plt.xlabel('\nBiome Definition Scheme', fontsize=80, fontweight='bold')
plt.ylabel('IAV Contribution (%)', fontsize=80, fontweight='bold')
plt.tick_params(axis='y', labelsize=80, width=10)
plt.yticks(fontsize=60, fontweight='bold')
plt.tick_params(axis='x', width=40)
plt.ylim((0, 80))
drylandBar = mpatches.Patch(facecolor='orange', label='Dryland', edgecolor='black')
tropicalBar = mpatches.Patch(facecolor='darkgreen', label='Tropical Forest', edgecolor='black')
plt.legend(handles=[drylandBar, tropicalBar], loc='upper center', fontsize=70)

#Adding bars to denote significance between the boxplots (significance calculated in the permutation testing file)
plt.text(1, max(all_dryland_IAV_KG.values())+3, 'A', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(2, max(all_tropical_IAV_KG.values())+3, 'B', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(3.5, max(all_dryland_IAV_MODIS.values())+3, 'C', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(4.5, max(all_tropical_IAV_MODIS.values())+3, 'B', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(6, max(all_dryland_IAV_ARID.values())+3, 'A', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(7, max(all_tropical_IAV_PANGEA.values())+3, 'D', dict(size=70, weight='bold'), horizontalalignment='center')
# Display the plot
plt.tight_layout()

In [ ]:
# Comparison between models of the different regions contributions to the IAV

# Fig-Size & bar spacing
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 6

# Köppen bars 
plt.bar(pos - 7*width/2, all_dryland_IAV_KG.values(), width=width, label="Köppen Dryland", color="darkorange")
plt.bar(pos - 5*width/2, all_tropical_IAV_KG.values(), width=width, label="Köppen\nTropical Forest", color="darkgreen")

# MODIS bars 
plt.bar(pos - width/2, all_dryland_IAV_MODIS.values(), width=width, label="MODIS Dryland", color="orange")
plt.bar(pos + width/2, all_tropical_IAV_MODIS.values(), width=width, label="MODIS\nTropical Forest", color="green")

# Aridity bars
plt.bar(pos + 5*width/2, all_dryland_IAV_ARID.values(), width=width, label="Aridity Index & Agro-Ecological Zone Dryland", color="gold")
plt.bar(pos + 7*width/2, all_tropical_IAV_PANGEA.values(), width=width, label="Aridity Index & Agro-Ecological Zone\nTropical Forest", color="limegreen")

#Adding the labels 
plt.xlabel('Model',fontsize=30)
plt.ylabel('IAV Contribution (%)',fontsize=30)
new_labels = []
for key in all_tropical_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.yticks(fontsize=25)
plt.legend(fontsize=18, ncols=3)
plt.ylim((0,70))

# Show plot
plt.show()


In [ ]:
#Just Koppen
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 2

# Köppen bars 
plt.bar(pos - width/2, all_dryland_IAV_KG.values(), width=width, label="Köppen Dryland", color="orange")
plt.bar(pos + width/2, all_tropical_IAV_KG.values(), width=width, label="Köppen Tropical Forest", color="darkgreen")

#Adding labels
plt.xlabel('Model',fontsize=30)
plt.ylabel('IAV Contribution (%)',fontsize=30)

new_labels = []
for key in all_tropical_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.ylim((0,70))
# Show plot
plt.show()


In [ ]:
#Just MODIS
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 2

# Köppen bars 
plt.bar(pos - width/2, all_dryland_IAV_MODIS.values(), width=width, label="MODIS Dryland", color="orange")
plt.bar(pos + width/2, all_tropical_IAV_MODIS.values(), width=width, label="MODIS Tropical Forest", color="darkgreen")

#Adding labels
plt.xlabel('Model',fontsize=30)
plt.ylabel('IAV Contribution (%)',fontsize=30)

new_labels = []
for key in all_tropical_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.ylim((0,70))
# Show plot
plt.show()


In [ ]:
#Just AI&AEZ
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 2

# Köppen bars 
plt.bar(pos - width/2, all_dryland_IAV_ARID.values(), width=width, label="Aridity Index & Agro-Ecological Zone Dryland", color="orange")
plt.bar(pos + width/2, all_tropical_IAV_PANGEA.values(), width=width, label="Aridity Index & Agro-Ecological Zone Tropical Forest", color="darkgreen")

#Adding labels
plt.xlabel('Model',fontsize=30)
plt.ylabel('IAV Contribution (%)',fontsize=30)

new_labels = []
for key in all_tropical_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.ylim((0,70))
# Show plot
plt.show()

In [ ]:
#This block handles the calculation of the various stats required for the rest of the analysis in other files 

#Median Values considering the absolute values
print("Median values")
print(f"Koppen dry: {round(stats.median(all_dryland_IAV_KG.values()), 2)}")
print(f"Koppen tropical: {round(stats.median(all_tropical_IAV_KG.values()), 2)}")
print(f"MODIS dry: {round(stats.median(all_dryland_IAV_MODIS.values()), 2)}")
print(f"MODIS tropical: {round(stats.median(all_tropical_IAV_MODIS.values()), 2)}")
print(f"AI dry: {round(stats.median(all_dryland_IAV_ARID.values()), 2)}")
print(f"PANGEA tropical: {round(stats.median(all_tropical_IAV_PANGEA.values()), 2)}")

# Calculating the IQR for the different boxplots
print("IQRs")
print(f"Koppen dry 25: {round(np.percentile(np.array(list(all_dryland_IAV_KG.values())), 25), 2)}")
print(f"Koppen dry 75: {round(np.percentile(np.array(list(all_dryland_IAV_KG.values())), 75), 2)}")

print(f"Koppen tropical 25: {round(np.percentile(np.array(list(all_tropical_IAV_KG.values())), 25), 2)}")
print(f"Koppen tropical 75: {round(np.percentile(np.array(list(all_tropical_IAV_KG.values())), 75), 2)}")

print(f"MODIS dry 25: {round(np.percentile(np.array(list(all_dryland_IAV_MODIS.values())), 25), 2)}")
print(f"MOIDS dry 75: {round(np.percentile(np.array(list(all_dryland_IAV_MODIS.values())), 75), 2)}")

print(f"MODIS tropical 25: {round(np.percentile(np.array(list(all_tropical_IAV_MODIS.values())), 25), 2)}")
print(f"MOIDS tropical 75: {round(np.percentile(np.array(list(all_tropical_IAV_MODIS.values())), 75), 2)}")

print(f"AI dry 25: {round(np.percentile(np.array(list(all_dryland_IAV_ARID.values())), 25), 2)}")
print(f"AI dry 75: {round(np.percentile(np.array(list(all_dryland_IAV_ARID.values())), 75), 2)}")

print(f"PANGEA tropical 25: {round(np.percentile(np.array(list(all_tropical_IAV_PANGEA.values())), 25), 2)}")
print(f"PANGEA tropical 75: {round(np.percentile(np.array(list(all_tropical_IAV_PANGEA.values())), 75), 2)}")

# Just listing the values of the dry and tropical regions to use in the bootstraping and permutation
#Converting IAV values to float lists
dryland_Koppen = []
for value in list(all_dryland_IAV_KG.values()):
    dryland_Koppen.append(float(value))
print(f"Dryland KG: {dryland_Koppen}")

tropical_Koppen = []
for value in list(all_tropical_IAV_KG.values()):
    tropical_Koppen.append(float(value))
print(f"Tropical KG: {tropical_Koppen}")

dryland_MODIS = []
for value in list(all_dryland_IAV_MODIS.values()):
    dryland_MODIS.append(float(value))
print(f"Dryland MODIS: {dryland_MODIS}")

tropical_MODIS = []
for value in list(all_tropical_IAV_MODIS.values()):
    tropical_MODIS.append(float(value))
print(f"Tropical MODIS: {tropical_MODIS}")

dryland_ARID = []
for value in list(all_dryland_IAV_ARID.values()):
    dryland_ARID.append(float(value))
print(f"Dryland ARID: {dryland_ARID}")

tropical_PANGEA = []
for value in list(all_tropical_IAV_PANGEA.values()):
    tropical_PANGEA.append(float(value))
print(f"Tropical PANGEA: {tropical_PANGEA}")

#Also getting the values for the 3 defintion average across the models for the drylands and tropical forests
dryland_combined_defs = []
tropical_combined_defs = []

for i in range(18):
    dryland_combined_defs.append((dryland_Koppen[i] + dryland_MODIS[i] + dryland_ARID[i]) / 3)
    tropical_combined_defs.append((tropical_Koppen[i] + tropical_MODIS[i] + tropical_PANGEA[i]) / 3)

print("\n Three defintion averages: ")
print(f"3 Def average Dryland median: {round(np.median(dryland_combined_defs), 2)}")
print(f"3 Def average dry 25: {round(np.percentile(np.array(dryland_combined_defs), 25), 2)}")
print(f"3 Def average dry 75: {round(np.percentile(np.array(dryland_combined_defs), 75), 2)}")
print(f"3 Def average Tropical median: {round(np.median(tropical_combined_defs), 2)}")
print(f"3 Def average tropical 25: {round(np.percentile(np.array(tropical_combined_defs), 25), 2)}")
print(f"3 Def average tropical 75: {round(np.percentile(np.array(tropical_combined_defs), 75), 2)}")

print(f"three_def_IAV_values_dryland = {dryland_combined_defs}")
print(f"three_def_IAV_values_tropical = {tropical_combined_defs}")


In [ ]:
# This block is concerned with the calculation of the different areas that the different defintions take up

#Calculating the Koppen classification areas
file_koppen = nc.Dataset("KG_360x720.nc", "r")
climate_type = file_koppen.variables['kg'][:]
file_koppen.close()
map_data = climate_type[:,:,:]
#0 and 1 is tropical forest and monsoon && 2 and 3 are svannas and 4 and 5 are the 2 semi-arid regions
tropic_climate = climate_type[:2, :, :] 
tropic_climate_2d = np.sum(tropic_climate, axis=0)
dryland_climate = climate_type[2:6, :, :] 
dryland_climate_2d = np.sum(dryland_climate, axis=0)

# Removing the land > 90% bare
file_ESACCI = nc.Dataset('ESACCI-LC-L4-LCCS-Map-300m-P1Y-aggregated-0.500000Deg-2019-v2.1.1.nc', 'r')
bare_soil = file_ESACCI.variables['Bare_soil'][:]
file_ESACCI.close()
dryland_climate_2d[bare_soil > 0.9] = 0 # Making the % of dryland or tropical for these regions == 0
tropic_climate_2d[bare_soil > 0.9] = 0

# Removing instances where less than 50% of dryland or tropical, as setting to zero
mask_50_tropic = tropic_climate_2d > 0.5
mask_50_dryland = dryland_climate_2d > 0.5
tropic_climate_2d[~mask_50_tropic] = 0
dryland_climate_2d[~mask_50_dryland] = 0

#tropical forest area Koppen
combined_maps = np.zeros((360,720))
combined_maps[tropic_climate_2d>0] = 1 #making the tropical region == 1 such that the multiplication with the area file gives area that can be divided from total
combined_maps[modis_data == 0] = 0
tropical_area_map = combined_maps*area_file
tropical_area = np.sum(tropical_area_map)
print(f"The Koppen tropical forest area: {tropical_area}")

#dry
combined_maps = np.zeros((360,720))
combined_maps[dryland_climate_2d>0] = 1 #repeating the process with the dryland areas
combined_maps[modis_data == 0] = 0
dryland_area_map = combined_maps*area_file
dryland_area = np.sum(dryland_area_map)
print(f"The dryland area: {dryland_area}")

#Other
combined_maps = np.zeros((360,720))
combined_maps[modis_data == 0] = 2
combined_maps[tropic_climate_2d>0] = 2
combined_maps[dryland_climate_2d>0] = 2
combined_maps[combined_maps == 0] = 1 #Setting to 1 the areas that are other land
combined_maps[combined_maps == 2] = 0 #Removing all that other area that is == 2
other_area_map = combined_maps*area_file
other_area = np.sum(other_area_map)
print(f"The other area using the Koppen definition: {other_area}")

total_surface_area = other_area+tropical_area+dryland_area #Getting the total surface are now that all of the areas have been grouped

#saving the area values for later use below
dryland_area_koppen = dryland_area/total_surface_area*100
tropical_area_koppen = tropical_area/total_surface_area*100
print(f"The proportion Koppen tropical forest: {round(tropical_area_koppen, 2)}")
print(f"The proportion Koppen dryland: {round(dryland_area_koppen, 2)}\n")

#Calculating the MODIS area now

# In the modis data 1 is the tropical and 3 is the dryland, 0 is the water
#Getting the tropical area
combined_maps = np.zeros((360,720))
combined_maps[modis_data == 1] = 1
tropical_area_map = combined_maps*area_file
tropical_area = np.sum(tropical_area_map)
print(f"The MODIS tropical forest area: {tropical_area}")

#Getting the dryland area
combined_maps = np.zeros((360,720))
combined_maps[modis_data == 3] = 1 #repeating the process with the dryland areas
dryland_area_map = combined_maps*area_file
dryland_area = np.sum(dryland_area_map)
print(f"The MODIS dryland area: {dryland_area}")

#Other area
combined_maps = np.zeros((360,720))
combined_maps[modis_data == 0] = 2
combined_maps[modis_data == 1] = 2
combined_maps[modis_data == 3] = 2
combined_maps[combined_maps == 0] = 1 #Setting to 1 the areas that are other land
combined_maps[combined_maps == 2] = 0 #Removing all that other area that is == 2
other_area_map = combined_maps*area_file
other_area = np.sum(other_area_map)
print(f"The MODIS other area: {other_area}")
total_surface_area = other_area+tropical_area+dryland_area #Summing the 3 different regions together

#saving the area values for later use below
dryland_area_moids = dryland_area/total_surface_area*100
tropical_area_modis= tropical_area/total_surface_area*100

print(f"The proportion MDOIS tropical forest: {round(tropical_area_modis, 2)}")
print(f"The proportion MODIS dryland: {round(dryland_area_moids, 2)}\n")

#Getting the areas of the regions for the AI & AEZ regions


# Only using the ARID range of arid to dry subhumid, excluding hyperarid as this will lead to inclusion of deserts which we are not doing
pangea_array = np.load('aez_tropical_PANGEA.npy')
arid_array = np.load("halfdeg_AI.npy")

# Making the PANGEA array all == 1 where there is actual tropical area
pangea_array[(pangea_array != 5) & (pangea_array != 6)& (pangea_array != 11) & (pangea_array != 12) & (pangea_array != 52)] = 0
pangea_array[pangea_array != 0] = 1

# Making the ARID array all == 2 where there is actual dryland area
arid_array[(0.05 < arid_array) & (arid_array< 0.65)]= -2
arid_array[arid_array != -2] = 0
arid_array[arid_array == -2] = 2
arid_array[300:,:] = 0 # Removing Antarctica.

# Puting the 2 arrays together by giving any overlapping priority to the tropical regions
combined_array = np.zeros((360,720))
combined_array[arid_array == 2] = 2 # Setting arid to 2
combined_array[pangea_array == 1] = 1 # setting tropical to 1
combined_array[(modis_data != 0) & (arid_array != 2) & (pangea_array != 1)] = 3 # Setting the other land to 3
combined_array[modis_data == 0] = 0

#Getting the tropical area
combined_tropical = np.zeros((360,720))
combined_tropical[combined_array == 1] = 1
tropical_area_map = combined_tropical*area_file
tropical_area = np.sum(tropical_area_map)
print(f"The AI & AEZ tropical forest area: {tropical_area}")

#Getting the dryland area
combined_dry = np.zeros((360,720))
combined_dry[combined_array == 2] = 1 #repeating the process with the dryland areas
dryland_area_map = combined_dry*area_file
dryland_area = np.sum(dryland_area_map)
print(f"The AI & AEZ dryland area: {dryland_area}")

#Other land
combined_maps = np.zeros((360,720))
combined_maps[combined_array == 3] = 1
other_area_map = combined_maps*area_file
other_area = np.sum(other_area_map)
print(f"The AI & AEZ other area: {other_area}")

#Putting the areas together for the 3 regions
total_surface_area = other_area+tropical_area+dryland_area

#saving the area values for later use below
dryland_area_ai_aez = dryland_area/total_surface_area*100
tropical_area_ai_aez = tropical_area/total_surface_area*100

print(f"The proportion AI & AEZ tropical forest: {round(tropical_area_ai_aez, 2)}")
print(f"The proportion AI & AEZ dryland: {round(dryland_area_ai_aez, 2)}")



In [ ]:
#This block is concrned with the plotting of the individual models area-weighted IAV contirbutions

#Using the computation of the areas from the above block to calculate the area-weighted IAV contributions
area_weighted_koppen_dryland = []
area_weighted_koppen_tropical = []
area_weighted_modis_dryland = []
area_weighted_modis_tropical = []
area_weighted_ai_aez_dryland = []
area_weighted_ai_aez_tropical = []

#Looping to calculate the area-weighted IAV values
for model in all_dryland_IAV_KG:
    area_weighted_koppen_dryland.append(all_dryland_IAV_KG[model] / dryland_area_koppen)
    area_weighted_koppen_tropical.append(all_tropical_IAV_KG[model] / tropical_area_koppen)
    area_weighted_modis_dryland.append(all_dryland_IAV_MODIS[model] / dryland_area_moids)
    area_weighted_modis_tropical.append(all_tropical_IAV_MODIS[model] / tropical_area_modis)
    area_weighted_ai_aez_dryland.append(all_dryland_IAV_ARID[model] / dryland_area_ai_aez)
    area_weighted_ai_aez_tropical.append(all_tropical_IAV_PANGEA[model] / tropical_area_ai_aez)

    
# Fig-Size & bar spacing
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 6

# Köppen bars 
plt.bar(pos - 7*width/2, area_weighted_koppen_dryland, width=width, label="Köppen Dryland", color="darkorange")
plt.bar(pos - 5*width/2, area_weighted_koppen_tropical, width=width, label="Köppen\nTropical Forest", color="darkgreen")

# MODIS bars 
plt.bar(pos - width/2, area_weighted_modis_dryland, width=width, label="MODIS Dryland", color="orange")
plt.bar(pos + width/2, area_weighted_modis_tropical, width=width, label="MODIS\nTropical Forest", color="green")

# Aridity bars
plt.bar(pos + 5*width/2, area_weighted_ai_aez_dryland, width=width, label="Aridity Index & Agro-Ecological Zone Dryland", color="gold")
plt.bar(pos + 7*width/2, area_weighted_ai_aez_tropical, width=width, label="Aridity Index & Agro-Ecological Zone\nTropical Forest", color="limegreen")

#Adding the labels 
plt.xlabel('Model',fontsize=30)
plt.ylabel('Area-Weighted IAV Contribution (%)',fontsize=30)
new_labels = []
for key in all_tropical_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.yticks(fontsize=25)
plt.legend(fontsize=18, ncols=3)


# Show plot
plt.show()

In [ ]:
#Making the area-weighted boxplot of the contributions

# Creating a list of all of the data
data = [area_weighted_koppen_dryland, area_weighted_koppen_tropical,
         area_weighted_modis_dryland, area_weighted_modis_tropical, area_weighted_ai_aez_dryland, area_weighted_ai_aez_tropical]

#Adjusting the figure size
plt.figure(figsize=(3939/100, 1593/100), dpi=100)

# Creating the boxplot
boxplots = plt.boxplot(data, positions=[1, 2, 3.5, 4.5, 6, 7], widths=0.9, patch_artist=True, medianprops=dict(color='black', linewidth=8), showmeans=True,
    meanprops={"marker": "D", "markerfacecolor": "grey", "markeredgecolor": "black", "markersize": 32},
    flierprops=dict( markersize=24, markeredgewidth=10), capprops=dict(linewidth=10), whiskerprops=dict(linewidth=10),  boxprops=dict(linewidth=10)
)

#Alternating colors
colors = ['orange', 'darkgreen', 'orange', 'darkgreen', 'orange', 'darkgreen']

boxes = boxplots['boxes']
whiskers = boxplots['whiskers']
caps = boxplots['caps']
fliers = boxplots['fliers']
medians = boxplots['medians']

i = 0
for box in boxes:
    box.set_facecolor('white')
    box.set_edgecolor(colors[i])
    i+=1

i = 0
for median in medians:
    median.set_color(colors[i])
    i+=1


i = 0
j=0
for whisker in whiskers:
    whisker.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

i = 0
j=0
for cap in caps:
    cap.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

fliers[0].set(markerfacecolor = 'white', markeredgecolor=colors[0])
fliers[4].set(markerfacecolor = 'white', markeredgecolor=colors[4])
fliers[5].set(markerfacecolor = 'white', markeredgecolor=colors[5])

#Adding labels
plt.xticks([1.5, 4, 6.5], ['Köppen', 'MODIS', 'Aridity Index &\nAgro-Ecological Zones'], fontsize=70)
plt.xlabel('\nBiome Definition Scheme', fontsize=80, fontweight='bold')
plt.ylabel('Area-Weighted\nIAV Contribution (%)', fontsize=80, fontweight='bold')
plt.tick_params(axis='y', labelsize=80, width=10)
plt.yticks(fontsize=60, fontweight='bold')
plt.tick_params(axis='x', width=40)
drylandBar = mpatches.Patch(facecolor='orange', label='Dryland', edgecolor='black')
tropicalBar = mpatches.Patch(facecolor='darkgreen', label='Tropical Forest', edgecolor='black')
plt.legend(handles=[drylandBar, tropicalBar], loc='upper center', bbox_to_anchor=(0.4, 1), fontsize=70)
plt.ylim(0, 5.5)
plt.yticks([0,1,2,3,4,5])
#Adding the lettering for significance calculated in permutation tests file
plt.text(1, max(area_weighted_koppen_dryland)+0.3, 'A', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(2, max(area_weighted_koppen_tropical)+0.3, 'B', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(3.5, max(area_weighted_modis_dryland)+0.3, 'A', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(4.5, max(area_weighted_modis_tropical)+0.3, 'B', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(6, max(area_weighted_ai_aez_dryland)+0.3, 'C', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(7, max(area_weighted_ai_aez_tropical)+0.3, 'B', dict(size=70, weight='bold'), horizontalalignment='center')
# Display the plot
plt.tight_layout()


In [ ]:
#This block is concenred with reporting the contirbution values of the "other biome" region not included within either the dryland or tropical 

print(f"Koppen other: {round(stats.median(all_other_IAV_KG.values()), 2)}")
print(f"Koppen other 25: {round(np.percentile(np.array(list(all_other_IAV_KG.values())), 25), 2)}")
print(f"Koppen other 75: {round(np.percentile(np.array(list(all_other_IAV_KG.values())), 75), 2)}")

print(f"MODIS other: {round(stats.median(all_other_IAV_MODIS.values()), 2)}")
print(f"Koppen other 25: {round(np.percentile(np.array(list(all_other_IAV_MODIS.values())), 25), 2)}")
print(f"Koppen other 75: {round(np.percentile(np.array(list(all_other_IAV_MODIS.values())), 75), 2)}")

print(f"AI&AEZ other: {round(stats.median(all_other_IAV_PA.values()), 2)}")
print(f"Koppen other 25: {round(np.percentile(np.array(list(all_other_IAV_PA.values())), 25), 2)}")
print(f"Koppen other 75: {round(np.percentile(np.array(list(all_other_IAV_PA.values())), 75), 2)}")
